<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Python: Projeto Final
Caderno de **Aula**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução ao Kaggle;</li>
  <li>Exploração de dados;</li>
  <li>Transformação e limpeza de dados;</li>
  <li>Visualização de dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Aulas**

## 1\. Introdução ao Kaggle

[Kaggle](https://www.kaggle.com/) é a maior comunidade online de ciência de dados e aprendizado de máquina. A plataforma permite que usuários encontrem e publiquem conjuntos de **dados**, construam e compartilhem **notebooks** (como este do Google Colab) e participem de **competições** (que pagam muito dinheiro as vezes) e desafios de dados. 

> Vamos publicar nosso notebook de exercícios na plataforma web do Kaggle para que você possa compartilhar tudo o que você aprendeu neste curso e compor o seu portfólio.

**DESCRIÇÃO DO PROBLEMA:**

A instituição financeira quer conceder mais oportunidades de emprestimos para os seus clientes, Porem é necessario verificar quais clientes sao compromissadas com a empresa, Por esse motivo vai ser feito uma analise dos dados com base em clientes adimplentes e inadimplentes. As seguintes etapas para a analise serão: (Exploraçao de Dados,Transformaçao e Limpeza de Dados e Visualização de Dados)

***IMPORTAÇOES DE BIBLIOTECAS**

In [ ]:
#import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt

## 2\. Exploração de Dados

Vamos explorar dados de crédito presentes neste neste [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv). Os dados estão no formato CSV e contém informações sobre clientes de uma instituição financeira. Em especial, estamos interessados em explicar a segunda coluna, chamada de **default**, que indica se um cliente é adimplente(`default = 0`), ou inadimplente (`default = 1`), ou seja, queremos entender o porque um cliente deixa de honrar com suas dívidas baseado no comportamento de outros atributos, como salário, escolaridade e movimentação financeira. Uma descrição completa dos atributos está abaixo.

> O atributo de interesse (`default`) é conhecido como **variável resposta** ou **variável dependente**, já os demais atributos que buscam explicá-la (`idade`, `salário`, etc.) são conhecidas como **variáveis explicatívas**, **variáveis independentes** ou até **variáveis preditoras**.



| Coluna  | Descrição |
| ------- | --------- |
| id      | Número da conta |
| default | Indica se o cliente é adimplente (0) ou inadimplente (1) |
| idade   | --- |
| sexo    | --- |
| depedentes | --- |
| escolaridade | --- |
| estado_civil | --- |
| salario_anual | Faixa do salario mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartao: blue, silver, gold e platinium |
| meses_de_relacionamento | Quantidade de meses desde a abertura da conta |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iteracoes com o cliente no último ano |
| meses_inatico_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite do cartão de crédito |
| valor_transacoes_12m | Soma total do valor das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |



Vamos começar lendos os dados num dataframe `pandas`.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values='na')

In [ ]:
df.head(n=10)

Com o dados em mãos, vamos conhecer um pouco melhor a estrutura do nosso conjunto de dados.

### **2.1. Estrutura** 

In [ ]:
df.shape # retorna uma tupla (qtd linhas, qtd colunas)

In [ ]:
df[df['default'] == 0].shape

In [ ]:
df[df['default'] == 1].shape

In [ ]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

In [ ]:
print(f"A proporcão clientes adimplentes é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A proporcão clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

### **2.2. Schema** 

In [ ]:
df.head(n=5)

 - Colunas e seus respectivos tipos de dados.

In [ ]:
df.dtypes

 - Atributos **categóricos**.

In [ ]:
df.select_dtypes('object').describe().transpose()

 - Atributos **numéricos**.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

### **2.3. Dados faltantes** 

Dados faltantes podem ser:

 - Vazios (`""`);
 - Nulos (`None`);
 - Não disponíveis ou aplicaveis (`na`, `NA`, etc.);
 - Não numérico (`nan`, `NaN`, `NAN`, etc).

In [ ]:
df.head()

Podemos verificar quais colunas possuem dados faltantes.

In [ ]:
df.isna().any()

 - A função abaixo levanta algumas estatisticas sobre as colunas dos dados faltantes.

In [ ]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)

In [ ]:
stats_dados_faltantes(df=df)

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 0])

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 1])

## 3\. Transformação e limpeza de dados

Agora que conhecemos melhor a natureza do nosso conjunto de dados, vamos conduzir uma atividade conhecida como *data wrangling* que consiste na transformação e limpeza dos dados do conjunto para que possam ser melhor analisados. Em especial, vamos remover:

 - Corrigir o *schema* das nossas colunas;
 - Remover os dados faltantes.

### **3.1. Correção de schema** 

Na etapa de exploração, notamos que as colunas **limite_credito** e **valor_transacoes_12m** estavam sendo interpretadas como colunas categóricas (`dtype = object`).

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].dtypes

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head(n=5)

Vamos criar uma função `lambda` para limpar os dados. Mas antes, vamos testar sua aplicação através do método funcional `map`:

In [ ]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))

valores_originais = ['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

Com a função `lambda` de limpeza pronta, basta aplica-la nas colunas de interesse.

In [ ]:
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(fn)
df['limite_credito'] = df['limite_credito'].apply(fn)

Vamos descrever novamente o *schema*:

In [ ]:
df.dtypes

 - Atributos **categóricos**.

In [ ]:
df.select_dtypes('object').describe().transpose()

 - Atributos **numéricos**.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

### **3.2. Remoção de dados faltantes** 

Como o pandas está ciente do que é um dados faltante, a remoção das linhas problemáticas é trivial.

In [ ]:
df.dropna(inplace=True)

Vamos analisar a estrutura dos dados novamente.

In [ ]:
df.shape

In [ ]:
df[df['default'] == 0].shape

In [ ]:
df[df['default'] == 1].shape

In [ ]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

In [ ]:
print(f"A proporcão adimplentes ativos é de {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes adimplentes é de {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print("")
print(f"A proporcão clientes inadimplentes é de {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"A nova proporcão de clientes inadimplentes é de {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

## 4\. Visualização de dados

Os dados estão prontos, vamos criar diversas visualizações para correlacionar variáveis explicativas com a variável resposta para buscar entender qual fator leva um cliente a inadimplencia. E para isso, vamos sempre comparar a base com todos os clientes com a base de adimplentes e inadimplentes.

Começamos então importando os pacotes de visualização e separando os clientes adimplentes e inadimplentes 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

In [ ]:
df_adimplente = df[df['default'] == 0]

In [ ]:
df_inadimplente = df[df['default'] == 1]

### **4.1. Visualizações categóricas** 

Nesta seção, vamos visualizar a relação entre a variável resposta **default** com os atributos categóricos.

In [ ]:
df.select_dtypes('object').head(n=5)

 - Escolaridade

In [ ]:
coluna = 'escolaridade'
titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes', 'Escolaridade dos Clientes Inadimplentes']


figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)
max_y = 0

for eixo,dataframe in enumerate([df, df_adimplente, df_inadimplente]):

    df_to_plot = dataframe[coluna].value_counts().reset_index()
    df_to_plot.columns = [coluna,'frequencia_absoluta']
    df_to_plot.sort_values(by=[coluna],inplace=True)
    
    f = sns.barplot(data=df_to_plot, x=coluna, y='frequencia_absoluta', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
    f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0,max_y))

    eixo += 1

figura.show()

****ESCOLARIDADE DOS CLIENTES:**
Ao verificar o grafico, existem mais cliente com certa gradução de mestrado com relação as outras tipos de educação formal, Porém pessoas sem educação formal se sobrepõem com mais pessoas com graduação e doutorado.Os que tem apenas o ensino médio é a segunda coluna que mais representa o quantidade de cliente que tem nesta instituição financeira. Vamos verificar como implica os defaut e nao default(clientes adimplentes e inadimplentes)

**ESCOLARIDADE DOS CLIENTES ADIMPLENTES:**
Os de mestrado representa muito mais cliente adimplentes.Entre os que nao tem educaçao formal e os que tem ensino médio mantém quase a mesma proporção tendo mais pessoas que são adimplentes com escolaridade de ensino medio completo. Porem, tem uma leve mudança de clientes adimplentes entre os que tem graduação comparado aos que tem doutorado, no primeiro grafico era quase a mesma proporção entre os de doutorado e graduação.Mas, as pessoas de graduação tem mais cliente adimplentes do que os que tem doutorado.

**ESCOLARIDADE DOS CLIENTES INADIMPLENTES:**
Os que são defaut apresentam com as mesmas proporçoes entre os tipos de escolaridade.Porém os que são de doutorado tiveram mais clientes inadimplentes comparado os que tem graduação. Mas essa leve mudança nao foi o suficente para tirar conclusões

 - Salário Anual

In [ ]:
coluna = 'salario_anual'
titulos = ['Salário Anual dos Clientes', 'Salário Anual dos Clientes Adimplentes', 'Salário Anual dos Clientes Inadimplentes']


figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)
max_y = 0

for eixo,dataframe in enumerate ([df, df_adimplente, df_inadimplente]):

  df_to_plot = dataframe[coluna].value_counts().reset_index()
  df_to_plot.columns = [coluna,'frequencia_absoluta']
  df_to_plot.sort_values(by=[coluna], inplace=True)

  f = sns.barplot(data=df_to_plot, x=coluna, y='frequencia_absoluta', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)
    
  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))
  eixo += 1

figura.show()

**SALARIO ANUAL DOS CLIENTES:**
Nesse grafico clientes que ganham anualmente menos que 40k são que mais representam metade dos clientes nesta instituição financeira,Logo vem os cliente que tem salario anual de 60k-40k com a segunda frenquencia mais alta com quase 1500, 60k-80k e 80k-120k mantém quase a mesma proporção de clientes com base nos seus salarios anuais.Mas, os que ganham mais do que 120k represantam quase 20% ou mais de clientes nessa empresa.

**SALARIO ANUAL DE CLIENTES ADIMPLENTES**
Diante desse grafico de clientes Adimplentes vimos que nao houve muita diferencia do primeiro grafico que representa todos os salarios anuais de clientes adimplentes e inadimplentes.Não houve mudanças significativas com base em dados de salarios anuais de clientes adimplentes

**SALARIO ANUAL DE CLIENTES INADIMPLENTES:**
Nesse granfico mantem a mesma proporção de salarios anuais de clientes adimplentes, não houve mudanças significaticas de clientes inadimplentes com base em seus salarios anuais. Porque ambos, os graficos de mantiveram a mesma proporção de clientes adimplentes e inadimplentes

In [ ]:
coluna = 'estado_civil'
titulos =['Estado Civil dos Clientes','Estado Civil dos Clientes Adimplentes','Estado Civil dos Clientes Inadimplentes']

figura, eixos = plt.subplots(1,3, figsize=(20,5), sharex = True)
max_y = 0

for eixo,dataframe in enumerate ([df,df_adimplente,df_inadimplente]):
    df_to_plot = dataframe[coluna].value_counts().reset_index()
    df_to_plot.columns = [coluna,'frequencia absoluta']
    df_to_plot.sort_values(by=[coluna],inplace = True)
    
    f = sns.barplot(data=df_to_plot,x=coluna,y='frequencia absoluta',ax=eixos[eixo])
    f.set(title =titulos[eixo], xlabel=coluna.capitalize(), ylabel = 'Frequencia Absoluta')
    
    f.set_xticklabels(labels= f.get_xticklabels(),rotation=90)
    
    _,max_y_f= f.get_ylim()
    may_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(max_y))
    eixo += 1
    
figura.show()

**ESTADO CIVIL DOS CLIENTES:**
Nesse Grafico vimos que temos muito mais clientes Casados do que divorciado e solteiro. Casados temos pouco mais de 3500 pessoas, divorciado temos de 500 a 600 pessoas e solteiro temos de 2500 a 2800 pessoas.


**ESTADO CIVIL DOS CLIENTES ADIMPLENTES:**
De clientes adimplentes os casados manteram uma quantidade de diferencia de 500 pessoas porque os clientes casados adimplentes atingiram 3000 pessoas consideradas adimplentes o divorciado nao teve muita diferencia quanse que mantende a mesma proporção e ja os solteiros temos a diferencia de quase 500 pessoas também. Diferencias no qual vemos de clientes inadimplentes 

**ESTADO CIVIL DOS CLIENTES INADIMPLENTES:**
Os clientes inadimplentes mantém a diferencia de 500 pessoas dos casados, pouca diferencia dos solteiros e mantendo ainda a mesma proporção dos divorciados. Vemos que os casados fazendo maior parte de clientes dessa instituição financeira os inadimplentes fazem a diferencia sobre o total atingindo quase o maximo do nosso primeiro grafico e a mesma coisa dos solteiro. Vemos aqui que podemos dizer que os defaut explica que os cientes fazem parte mais dos que sao casados e solteiro. Por algum motivo esses clientes se tornam inadimplentes quando sao casados ou solteiros, e o curioso que apesar de nao ter muito clientes com estado civil divorciado mantiveram a mesma proporção de inadimplentes e adimplentes.

### **4.2. Visualizações numéricas** 

Nesta seção, vamos visualizar a relação entre a variável resposta **default** com os atributos numéricos.

In [ ]:
df.drop(['id', 'default'], axis=1).select_dtypes('number').head(n=5)

 - Quantidade de Transações nos Últimos 12 Meses

In [ ]:
coluna = 'qtd_transacoes_12m'
titulos = ['Qtd. de Transações no Último Ano', 'Qtd. de Transações no Último Ano de Adimplentes', 'Qtd. de Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    _, max_y_f = f.get_ylim()
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0, max_y))

    eixo += 1

figura.show()

**QUANTIDADE DE TRANSAÇOES DO ULTIMO ANO:**
Nesse primeiro grafico ah dois picos de quantidade de transaçoes feitas nos ultimos doze meses, o primeiro é a quantidade entre 20 e 40 transaçoes feitas e outro fica entre 60 e 80 transaçoes feitas durante os 12 meses. Tendo os mais elevados entre 400 e pouco mais de 500 clientes que fizeram transações durante esses doze meses.

**QUANTIDADE DE TRANSAÇOES NO ULTIMO ANO DE ADIMPLENTES:**
Mantém quase a mesma proporção mas com uma certa diferençia mostrando um pico mais alto entre 60,80 e 100. Já da para analisar que os clientes adimplentes teve uma certa quantidade maior de transaçoes durante 1 ano.

**QUANTIDADE DE TRANSAÇOES NO ULTIMO ANO DE INADIMPLENTES:**
Os clientes inadimplentes apresentaram um pico mais alto entre 20,40 e 60 e ao ver o grafico da para perceber um certo complemento no eixo X que apresenta a quantidade de transaçoes nos ultimos 12 meses no grafico de clientes adimplentes. Quantidade feita entre 20,40,60 representa bem os clientes inadimplentes porque justo nesses valores que teve um certo pico de transaçoes no primeiro grafico e o segundo. Ja da para perceber que os clientes se tornaram inadimplentes porque a quantidade de transaçoes apesar de ser ainda menor tiveram que por algum motivo deixar de pagar os emprestimos em dia. 

 - Valor das Transações nos Últimos 12 Meses

In [ ]:
coluna = 'valor_transacoes_12m'
titulos = ['Valor das Transações no Último Ano', 'Valor das Transações no Último Ano de Adimplentes', 'Valor das Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

**VALOR DAS TRANSAÇOES NO ULTIMO ANO:**
Os valores das trasnsaçoes tiveram um grande pico mais ou menos 1000 reais a 5000, houve também transaçoes altas entre 7500 a 10000 e 12500 a 15000, nesse valores esta todos os clientes adimplentes e inadimplentes.

**VALOR DAS TRANSAÇOES NO ULTIMO ANO DE ADIMPLENTES:**
Os clientes adimplentes fizeram transaçoes com os seus picos de mais ou menos a 1000 a 5000, houve também transaçoes de valores altos nos 7500 e quase nos 100000 reais e manteve a mesma proporção dos 12500 a 175000 nos ultimos doze meses. Os clientes adimplentes de fazerem grandes transaçoes de valores altos ainda mantém como clientes adimplentes pagando todos emprestimos em dia. 

**VALOR DAS TRANSAÇOES NO ULTIMO ANO DE INADIMPLENTES:**
Porém tem uma grande diferençia nos clientes inadimplentes, no granfico mostra uma certo pico no valor das transaçoes entre mais ou menos 1000 ate 2500 e as transações mais altas estão entre 7500 a quase 10000. Nesse pico de 2500 mostrar que apesar de seram valores de transações menores, se torna preocupante porque esses clientes no futuro poderão nao ter a opotunidade fazer trasaçoes mais altas com mais frequençia. 

 - Valor de Transações nos Últimos 12 Meses x Quantidade de Transações nos Últimos 12 Meses

In [ ]:
f = sns.relplot(x='valor_transacoes_12m', y='qtd_transacoes_12m', data=df, hue='default')
_ = f.set(
    title='Relação entre Valor e Quantidade de Transações no Último Ano', 
    xlabel='Valor das Transações no Último Ano', 
    ylabel='Quantidade das Transações no Último Ano'
  )

**RELAÇOES ENTRE VALOR E QUANTIDADE DE TRANSAÇOES DO ULTIMO ANO:**
Na relação de clientes adimplentes e inadimplentes dá para perceber que os clientes inadimplentes apresentam quantidade relevantes entre 20,40 e 60 nos valores de mais ou menos 1000 a 5000 reais, e nas transaçoes mais altas esta entre 7500 e pouco mais de 10000 reais e nessas transaçoes altas apresentam poucas quantidade  que fica entre 60 e pouco mais de 80. Fica claro que mais vez os valores e quantidade de transaçoes sao pequenas mas, que representa bem os clientes inadimplentes. Por isso nesse periodo é necessario analisar e buscar soluçoes que possam verificar o porque que esses clientes se tornam inadimplentes nesses periodos durante 12 meses.

**RELAÇOES ENTRE VALOR E QUANTIDADE DE TRANSAÇOES DO ULTIMO ANO:**
Os clientes adimplentes eles fazem as suas transaçoes com mais frenquencia em valores pequenos de 2500 ate 7500. Porém da para ver que os cliente fazem trasaçoes com valores mais altos e com mais frenquencia, e essa parte apresentou bem a quantidade de clientes que fazem parte dessa estatistica (VALORES ALTOS COM MAIS FRENQUENCIA NAS SUAS TRANSAÇOES) esses clientes podem representar uma grande parte de pessoas financeiramente classe media e alta.

**INSIGHT FINAL RESUMO:**
Nas visualizaçoes categoricas vemos que na escolaridade e salario anual mantiveram a mesma proporção de clientes adimplentes e inadimplentes nao explicando o motivo do defaut. Mas os clientes de estado Civil mostraram alguma diferencia de clientes adimplentes e inadimplentes, apesar de que as intituição financeira tenha mais clientes de estado civil casados e solteiro tiveram uma relevancia de 500 pessoas de adimplentes e inadimplentes, o que é uma grande diferencia porque ambos complementam os totais de clientes que fazem parte da intituição financeira. Nas visualizaçoes numericas as quantidades de transaçoes e os valores das transaçoes explicaram o default. A quantidade de transaçoes se concentrava de clientes inadimplentes de 20,40,60 no grafico enquanto no valor em relação a quantidade os valores ficavam entre 2500 a 10000 reais. O que pode ser nesses dados que no meio de quantidade e Valor em visualizaçoes numericas possam estar clientes casados e solteiros. Concluindo a intuição financeira deve entrar em contato no periodo que mais os clientes fazem transaçoes entre valor e quantidade de clientes inadimplentes com estado civil solteiro ou casado e descobrir o motivo de porque estarem inadimplentes e assim descobrir a melhor forma possivel de resolver o problema.

## 5\. Storytelling

O *storytelling* no contexto de dados é um técnica de apresentação de resultados orientado a dados, ou seja, contar uma história baseada nos *insights* que foram gerados através da análise dos dados. Notebooks como este do Google Colab e os do Kaggle são excelentes ferramentas para conduzir e compartilhar *storytelling* de dados devido a natureza texto-código de suas céluas.

Para você montar o seu portifólio, eu sugiro a construção de um notebook com a seguinte estrutura (vou disponibiliza-la nos exercícios):

 1. Título;
 2. Breve descrição do problema;
 3. Código de importação de bibliotecas;
 4. Código o download/carregamento/geração de dados;
 5. Etapa de exploração;
 6. Etapa de limpeza e transformação;
 7. Etapa de análise (com visualizações);
 8. Resumo dos *insights* gerados.

Busquei organizar este notebook desta forma. Ademais, os notebooks presentes na plataforma do Kaggle são excelentes exemplos a serem seguidos, em especial os primeiros colocados em competições.

Para finalizar, algumas dicas:

 1. Estruture seu código sempre de acordo com as boas práticas PEP8, assim ele será mais legível para o leitor;
 2. Sempre se preocupe com a aparência dos seus gráficos, todos devem ter (no mínimo) título no topo e nos eixos;
 3. Use e abuso das células de texto para estruturar seu notebook, siga as mesmas técnicas que eu utilizo nos notebooks do curso para estruturar seu texto.

____

Quando terminar, se possível, entra em contato comigo, eu adoraria ver o seu notebook final pronto! Meu LinkedIn está no topo da página, logo após o logo da EBAC. Espero que você tenha gostado, foi um prazer trabalhar com você!

Até a próxima.